In [1]:
#Import packages
import requests
import os
import webbrowser
import pandas as pd
import numpy as np
import sqlite3
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import matplotlib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

In [2]:
#load the data

ticker = pd.read_csv("C:\\Users\\yiwent\\Desktop\\ResearchAssistant\\tickerreal.csv", header = None)
ticker.columns = ['Ticker']

In [3]:
#By observation, get the list of websites containing the info of the companies specified by each ticker

crawl_list = []
for tickers in ticker['Ticker']:
    crawl_list.append('https://www.sec.gov/cgi-bin/browse-edgar?CIK=' + tickers + "&owner=exclude&action=getcompany&Find=Search")
crawl_list

['https://www.sec.gov/cgi-bin/browse-edgar?CIK=AHG&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=AIF&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=AMIC&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=BBHL&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=BH&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=BYI&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=C&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=CALC&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=CCBC&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=CHMT&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bi

In [4]:
####PART I#####

#Take a look at the structure of HTML

url = "https://www.sec.gov/cgi-bin/browse-edgar?CIK=VXRT&owner=exclude&action=getcompany"
url_get = requests.get(url)
soup = BeautifulSoup(url_get.content, 'lxml')
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="ENG">
<head>
<title>EDGAR Search Results</title>
<link href="/include/interactive.css" rel="stylesheet" type="text/css"/>
<link href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0000072444&amp;type=&amp;dateb=&amp;owner=exclude&amp;count=40&amp;output=atom" rel="alternate" title="ATOM" type="application/atom+xml"/>
</head>
<body style="margin: 0">
<!-- SEC Web Analytics - For information please visit: https://www.sec.gov/privacy.htm#collectedinfo -->
<noscript><iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-TD3BKV" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.ins

In [5]:
soup_maininfo = soup.find("div", {"class": "companyInfo"})
soup_compName = soup_maininfo.find("span",{"class": "companyName"})
soup_compName

<span class="companyName">Vaxart, Inc. <acronym title="Central Index Key">CIK</acronym>#: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0000072444&amp;owner=exclude&amp;count=40">0000072444 (see all company filings)</a></span>

In [6]:
#we want to get the name first, find out the redundant text

soup_redundant = soup_compName.find("a")
soup_redundant.text

'0000072444 (see all company filings)'

In [7]:
# Find the firm name  (Note: the string "CIK#: " is the same for HTML)

print(soup_compName.text.replace(" CIK#: "+ soup_redundant.text,""))

Vaxart, Inc.


In [8]:
#CIK of the firm (Note: removed the redundant text that appears in every HTML)

print(soup_redundant.text.replace(" (see all company filings)",""))

0000072444


In [9]:
#Find the SIC

soup_maininfo.find_all('a')[1].text

'2836'

In [10]:
#Take a look at class companyInfo

soup_maininfo

<div class="companyInfo">
<span class="companyName">Vaxart, Inc. <acronym title="Central Index Key">CIK</acronym>#: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0000072444&amp;owner=exclude&amp;count=40">0000072444 (see all company filings)</a></span>
<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=2836&amp;owner=exclude&amp;count=40">2836</a> - BIOLOGICAL PRODUCTS (NO DIAGNOSTIC SUBSTANCES)<br/>State location: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;State=CA&amp;owner=exclude&amp;count=40">CA</a> | State of Inc.: <strong>DE</strong> | Fiscal Year End: 1231<br/>formerly: Aviragen Therapeutics, Inc. (filings through 2018-02-15)<br/>formerly: Biota Pharmaceuticals, Inc. (filings through 2016-04-13)<br/>formerly: NABI  BIOPHARMACEUTICALS (filings through 2010-06-29)<br/>formerly: NABI /DE/ (filings through 2002-02-14)<br/>formerly: NABI BIOPHARMACEUTICALS (filings through 2012-1

In [11]:
#Replace "<br>" using a symbol, help us to find location

for br in soup_maininfo.find_all("br"):
    br.replace_with(" $")

In [12]:
#This is our formal name locator

x = soup_maininfo.text.find("$f")
y = soup_maininfo.text.find("$(A")
x,y

(180, 566)

In [13]:
#Take a look if the range is correct

formal = soup_maininfo.text[180:566]
formal

'$formerly: Aviragen Therapeutics, Inc. (filings through 2018-02-15) $formerly: Biota Pharmaceuticals, Inc. (filings through 2016-04-13) $formerly: NABI  BIOPHARMACEUTICALS (filings through 2010-06-29) $formerly: NABI /DE/ (filings through 2002-02-14) $formerly: NABI BIOPHARMACEUTICALS (filings through 2012-11-08) $formerly: NORTH AMERICAN BIOLOGICALS INC (filings through 1996-02-14) '

In [14]:
#Replace "$formally: " with an astro the help us break the formal name

formal = formal.replace("$formerly: ","*")
formal

'*Aviragen Therapeutics, Inc. (filings through 2018-02-15) *Biota Pharmaceuticals, Inc. (filings through 2016-04-13) *NABI  BIOPHARMACEUTICALS (filings through 2010-06-29) *NABI /DE/ (filings through 2002-02-14) *NABI BIOPHARMACEUTICALS (filings through 2012-11-08) *NORTH AMERICAN BIOLOGICALS INC (filings through 1996-02-14) '

In [15]:
#Split the name

formal = formal.split("*")
formal

['',
 'Aviragen Therapeutics, Inc. (filings through 2018-02-15) ',
 'Biota Pharmaceuticals, Inc. (filings through 2016-04-13) ',
 'NABI  BIOPHARMACEUTICALS (filings through 2010-06-29) ',
 'NABI /DE/ (filings through 2002-02-14) ',
 'NABI BIOPHARMACEUTICALS (filings through 2012-11-08) ',
 'NORTH AMERICAN BIOLOGICALS INC (filings through 1996-02-14) ']

In [16]:
#This tells us it has len - 1 formal filing name in total (index: 1 --> len(formal)-1)
for i in range(1,len(formal)):
    print(formal[i])

Aviragen Therapeutics, Inc. (filings through 2018-02-15) 
Biota Pharmaceuticals, Inc. (filings through 2016-04-13) 
NABI  BIOPHARMACEUTICALS (filings through 2010-06-29) 
NABI /DE/ (filings through 2002-02-14) 
NABI BIOPHARMACEUTICALS (filings through 2012-11-08) 
NORTH AMERICAN BIOLOGICALS INC (filings through 1996-02-14) 


In [17]:
####PART II#####

#Now we examine what we should do if the Ticker cannont be located#

#Use CIK = AIF

bad_url = "https://www.sec.gov/cgi-bin/browse-edgar?CIK=AIF&owner=exclude&action=getcompany"
bad_url_get = requests.get(bad_url)
bad_soup = BeautifulSoup(bad_url_get.content, 'lxml')
bad_soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<html lang="ENG">
<head>
<title>Company Information: </title>
<!-- BEGIN HEADER -->
<script language="JavaScript" src="/include/sec.js" type="text/javascript"></script>
</head>
<body style="margin: 0">
<!-- SEC Web Analytics - For information please visit: https://www.sec.gov/privacy.htm#collectedinfo -->
<noscript><iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-TD3BKV" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-TD3BKV');</script>
<!-- End SEC Web Analytics -->
<table bgcolor="white" border="0" cellpadding="0" cellspacing="0" summary="" width="100%">
<t

In [18]:
# we can locate this ticker, the title is EDGAR Search Result
# Otherwise, we only get "Company Information: " in our head

#soup.head.text
bad_soup.head.text.replace("\n","")

'Company Information: '

In [19]:
####PART III#####

# Now we examine if the company has no formal name: JAX

nof_url = "https://www.sec.gov/cgi-bin/browse-edgar?CIK=JAX&owner=exclude&action=getcompany"
nof_url_get = requests.get(nof_url)
nof_soup = BeautifulSoup(nof_url_get.content, 'lxml')
nof_soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="ENG">
<head>
<title>EDGAR Search Results</title>
<link href="/include/interactive.css" rel="stylesheet" type="text/css"/>
<link href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0001617227&amp;type=&amp;dateb=&amp;owner=exclude&amp;count=40&amp;output=atom" rel="alternate" title="ATOM" type="application/atom+xml"/>
</head>
<body style="margin: 0">
<!-- SEC Web Analytics - For information please visit: https://www.sec.gov/privacy.htm#collectedinfo -->
<noscript><iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-TD3BKV" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.ins

In [20]:
#Take a look of the class and look for differences

nof_soup_maininfo = nof_soup.find("div", {"class": "companyInfo"})
nof_soup_maininfo

<div class="companyInfo">
<span class="companyName">J. Alexander's Holdings, Inc. <acronym title="Central Index Key">CIK</acronym>#: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0001617227&amp;owner=exclude&amp;count=40">0001617227 (see all company filings)</a></span>
<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=5812&amp;owner=exclude&amp;count=40">5812</a> - RETAIL-EATING PLACES<br/>State location: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;State=TN&amp;owner=exclude&amp;count=40">TN</a> | State of Inc.: <strong>TN</strong> | Fiscal Year End: 1230<br/>(Assistant Director Office: 5)<br/>Get <a href="/cgi-bin/own-disp?action=getissuer&amp;CIK=0001617227"><b>insider transactions</b></a> for this <b>issuer</b>.
</p>
</div>

In [21]:
#We can see that we cannot find "$f" if a company dont have formal mame

for br in nof_soup_maininfo.find_all("br"):
    br.replace_with(" $")
    
x = nof_soup_maininfo.text.find("$f")
y = nof_soup_maininfo.text.find("$(A")
x,y

(-1, 171)

In [331]:
# Where the game begins(test with first five ticker)

In [22]:
#Test Round
test_list = crawl_list[0:5]
test_list

['https://www.sec.gov/cgi-bin/browse-edgar?CIK=AHG&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=AIF&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=AMIC&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=BBHL&owner=exclude&action=getcompany&Find=Search',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=BH&owner=exclude&action=getcompany&Find=Search']

In [23]:
#First we create a dataframe with Ticker,Name,CIK,SIK

list_name=[]
list_CIK=[]
list_SIK=[]


for url in test_list:
    url_get = requests.get(url)
    soup = BeautifulSoup(url_get.content, 'lxml')
    if soup.head.text.replace("\n","") == "Company Information: ":
        list_name.append("Unlocated")
        list_CIK.append("Unlocated")
        list_SIK.append("Unlocated")
    else:
        soup_maininfo = soup.find("div", {"class": "companyInfo"})
        soup_compName = soup_maininfo.find("span",{"class": "companyName"})
        soup_redundant = soup_compName.find("a")
        name = soup_compName.text.replace(" CIK#: "+ soup_redundant.text,"")
        CIK = soup_redundant.text.replace(" (see all company filings)","")
        SIK = soup_maininfo.find_all('a')[1].text
        list_name.append(name)
        list_CIK.append(CIK)
        list_SIK.append(SIK)

In [24]:
temp = ticker[0:5]
temp.assign(**{'Name': list_name, 'CIK': list_CIK, 'SIK': list_SIK}) #looks good

,Ticker,Name,CIK,SIK
0,AHG,APRIA HEALTHCARE GROUP INC,0000882289,8082
1,AIF,Unlocated,Unlocated,Unlocated
2,AMIC,"AMIC Holdings, Inc.",0000097196,6321
3,BBHL,B B Holdings LTD,0000882505,7340
4,BH,Biglari Holdings Inc.,0001726173,5812


In [25]:
# now we create another table with Ticker and Formal Names

list_ticker=[]
list_Formally=[]

for tickers in ticker['Ticker'][0:5]:
    url = ('https://www.sec.gov/cgi-bin/browse-edgar?CIK=' + tickers + "&owner=exclude&action=getcompany&Find=Search")
    url_get = requests.get(url)
    soup = BeautifulSoup(url_get.content, 'lxml')
    if soup.head.text.replace("\n","") == "Company Information: ":
        list_ticker.append(tickers)
        list_Formally.append("Unlocated")
        continue
    soup_maininfo = soup.find("div", {"class": "companyInfo"})    
    for br in soup_maininfo.find_all("br"):
        br.replace_with(" $")
    if soup_maininfo.text.find("$f") == -1:
        list_ticker.append(tickers)
        list_Formally.append("No Formal Names")
    else:
        formal = soup_maininfo.text[soup_maininfo.text.find("$f"):soup_maininfo.text.find("$(A")]
        formal = formal.replace("$formerly: ","*")
        formal = formal.split("*")
        for i in range(1,len(formal)):
            list_ticker.append(tickers)
            list_Formally.append(formal[i])

In [26]:
table_temp = pd.DataFrame({'Ticker':list_ticker, 'Formally': list_Formally})
table_temp #looks good

,Ticker,Formally
0,AHG,ABBEY HEALTHCARE GROUP INC/DE (filings through...
1,AIF,Unlocated
2,AMIC,AMERICAN INDEPENDENCE CORP (filings through 20...
3,AMIC,SOFTNET SYSTEMS INC (filings through 2002-11-19)
4,BBHL,BHI CORPORATION (filings through 1999-02-11)
5,BBHL,CARLISLE HOLDINGS LTD (filings through 2006-03...
6,BH,NBHSA Inc. (filings through 2018-04-02)


In [27]:
# Now we can start building our first table(Table1 contains: Ticker|Name|CIK|SIK )

list_name=[]
list_CIK=[]
list_SIK=[]


for url in crawl_list:
    url_get = requests.get(url)
    soup = BeautifulSoup(url_get.content, 'lxml')
    if soup.head.text.replace("\n","") == "Company Information: ":
        list_name.append("Unlocated")
        list_CIK.append("Unlocated")
        list_SIK.append("Unlocated")
    else:
        soup_maininfo = soup.find("div", {"class": "companyInfo"})
        soup_compName = soup_maininfo.find("span",{"class": "companyName"})
        soup_redundant = soup_compName.find("a")
        name = soup_compName.text.replace(" CIK#: "+ soup_redundant.text,"")
        CIK = soup_redundant.text.replace(" (see all company filings)","")
        SIK = soup_maininfo.find_all('a')[1].text
        list_name.append(name)
        list_CIK.append(CIK)
        list_SIK.append(SIK)

In [43]:
Table1 = ticker
Table1 = Table1.assign(**{'Name': list_name, 'CIK': list_CIK, 'SIK': list_SIK})
Table1

,Ticker,Name,CIK,SIK
0,AHG,APRIA HEALTHCARE GROUP INC,0000882289,8082
1,AIF,Unlocated,Unlocated,Unlocated
2,AMIC,"AMIC Holdings, Inc.",0000097196,6321
3,BBHL,B B Holdings LTD,0000882505,7340
4,BH,Biglari Holdings Inc.,0001726173,5812
5,BYI,"BALLY TECHNOLOGIES, INC.",0000002491,7372
6,C,CITIGROUP INC,0000831001,6021
7,CALC,CALIFORNIA COASTAL COMMUNITIES INC,0000840216,1531
8,CCBC,Unlocated,Unlocated,Unlocated
9,CHMT,LANXESS Solutions US Inc.,0001091862,2820


In [44]:
#Number of Unlocated Tickers = 14

Table1['Name'].value_counts()[0]

14

In [45]:
#We store the csv file of Table 1 just in case

#Since for CIK 0 can be leading digits, we need to change table 1 a bit so that it shows nicely on Excel

#Table1['CIK'] = '="' + Table1['CIK'] + '"'
#Table1.to_csv("C:\\Users\\yiwent\\Desktop\\ResearchAssistant\\table1.csv",index = False)

In [46]:
# Now we start building Table2(Table1 contains: Ticker|Formally )

list_ticker=[]
list_Formally=[]

for tickers in ticker['Ticker']:
    url = ('https://www.sec.gov/cgi-bin/browse-edgar?CIK=' + tickers + "&owner=exclude&action=getcompany&Find=Search")
    url_get = requests.get(url)
    soup = BeautifulSoup(url_get.content, 'lxml')
    if soup.head.text.replace("\n","") == "Company Information: ":
        list_ticker.append(tickers)
        list_Formally.append("Unlocated")
        continue
    soup_maininfo = soup.find("div", {"class": "companyInfo"})    
    for br in soup_maininfo.find_all("br"):
        br.replace_with(" $")
    if soup_maininfo.text.find("$f") == -1:
        list_ticker.append(tickers)
        list_Formally.append("No Formal Names")
    else:
        formal = soup_maininfo.text[soup_maininfo.text.find("$f"):soup_maininfo.text.find("$(A")]
        formal = formal.replace("$formerly: ","*")
        formal = formal.split("*")
        for i in range(1,len(formal)):
            list_ticker.append(tickers)
            list_Formally.append(formal[i])

In [47]:
table2 = pd.DataFrame({'Ticker':list_ticker, 'Formally': list_Formally})
table2

,Ticker,Formally
0,AHG,ABBEY HEALTHCARE GROUP INC/DE (filings through...
1,AIF,Unlocated
2,AMIC,AMERICAN INDEPENDENCE CORP (filings through 20...
3,AMIC,SOFTNET SYSTEMS INC (filings through 2002-11-19)
4,BBHL,BHI CORPORATION (filings through 1999-02-11)
5,BBHL,CARLISLE HOLDINGS LTD (filings through 2006-03...
6,BH,NBHSA Inc. (filings through 2018-04-02)
7,BYI,ALLIANCE GAMING CORP (filings through 2006-03-...
8,C,TRAVELERS GROUP INC (filings through 1998-09-10)
9,C,TRAVELERS INC (filings through 1995-01-30)


In [49]:
#Again, store table 2 just in case

#table2.to_csv("C:\\Users\\yiwent\\Desktop\\ResearchAssistant\\table2.csv",index = False)

In [51]:
#OK finally merge the two table

final_table = pd.merge(Table1, table2, on="Ticker", how = "right")
final_table

,Ticker,Name,CIK,SIK,Formally
0,AHG,APRIA HEALTHCARE GROUP INC,"=""0000882289""",8082,ABBEY HEALTHCARE GROUP INC/DE (filings through...
1,AIF,Unlocated,"=""Unlocated""",Unlocated,Unlocated
2,AMIC,"AMIC Holdings, Inc.","=""0000097196""",6321,AMERICAN INDEPENDENCE CORP (filings through 20...
3,AMIC,"AMIC Holdings, Inc.","=""0000097196""",6321,SOFTNET SYSTEMS INC (filings through 2002-11-19)
4,BBHL,B B Holdings LTD,"=""0000882505""",7340,BHI CORPORATION (filings through 1999-02-11)
5,BBHL,B B Holdings LTD,"=""0000882505""",7340,CARLISLE HOLDINGS LTD (filings through 2006-03...
6,BH,Biglari Holdings Inc.,"=""0001726173""",5812,NBHSA Inc. (filings through 2018-04-02)
7,BYI,"BALLY TECHNOLOGIES, INC.","=""0000002491""",7372,ALLIANCE GAMING CORP (filings through 2006-03-...
8,C,CITIGROUP INC,"=""0000831001""",6021,TRAVELERS GROUP INC (filings through 1998-09-10)
9,C,CITIGROUP INC,"=""0000831001""",6021,TRAVELERS INC (filings through 1995-01-30)


In [52]:
## Final Output

final_table.to_csv("C:\\Users\\yiwent\\Desktop\\ResearchAssistant\\final_table.csv",index = False)

In [ ]:
##IMPORTANT NOTE!!!###

#After examining the final csv file, it seems that the company whose Ticker is "NEWT" doesn't have a SIC number.

#Solution: since it is the only case and the info is not on the website, I manually editted its SIC# to "No SIC"

In [ ]:
###Thank you####